In [10]:
import tensorflow as tf
import numpy as np
import os
import time
#%config Completer.use_jedi = False
#tf.autograph.set_verbosity(3, True)


from tensorflow import keras
from tensorflow.keras import layers
tf.__version__

'2.4.1'

In [11]:
text = np.load("fish_murme2.npy")
print(text.shape)
# The unique characters in the file
vocabulary = sorted(set(text))
print ('{} unique characters.'.format(len(vocabulary)))

vocabulary_size = len(vocabulary)

c2emb = {char:index for index, char in enumerate(vocabulary)}
emb2c = np.array(vocabulary)

(12001,)
512 unique characters.


In [12]:
sequence_length = 500
examples_per_epoch = len(text)//sequence_length
text_emb = np.array([c2emb[code] for code in text])
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_emb)

In [13]:
sequences = char_dataset.batch(sequence_length+1, drop_remainder=True)

for item in sequences.take(1):
      print(item.numpy())

[390 395 395 395 395 393 393 250 250 250 250 250 250 252 252 252 256 251
 252 258 252 258 240 387 372 214  42  42 216 217  26  30  30 184 196 177
 345 180 145 144 145 145 328 325 345 345 143 143 143 143   4 177 143 142
 320 444 442 318 318 442 448 448 324 324 320 324 137 141 141 320 444 444
 334 323 323 137 137 324 324 320 136 137 141 189 355 356 373  92  96  99
  99 247 247 248 251 251 245 256 256 103 256 256 398 486 394 128 253 251
 251 252 395 252 103 252 487 483 101 103  93  90  94  11  13 257 484 484
 395 395 395 395 395 395 395 403 403 483 464 487 484 398 398 398 394 398
 331 330 395 252 159 330 252 256 256 252 252 256 252 252 267 265 158 156
 281 282 117 117 289 289 285 290 290 290 286 132 311 311 433 433 434 312
 312 317 317 171 337 446 447 443 448 444 448 448 448 448 443 326 329 329
 329 329 451 448 449 449 444 325 318 448 448 319 324 324 448 320 324 324
 320 324 324 320 141 141 137 137 137 137 141 141 141 176 343 349 459 459
 460 461 460 461 350 350 347 350 350 350 347 347 34

In [5]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [6]:
# Batch size
batch = 20
print(examples_per_epoch)
steps_per_epoch = examples_per_epoch//batch

# TF data maintains a buffer in memory in which to shuffle data
# since it is designed to work with possibly endless data
buffer = 10000

dataset = dataset.shuffle(10000).batch(batch_size=batch, drop_remainder=True)

#dataset = dataset.repeat()
for a in dataset.take(1):
    print(a)

24
(<tf.Tensor: shape=(20, 500), dtype=int64, numpy=
array([[480, 380, 229, ..., 172,  23,  22],
       [230, 230, 230, ..., 391, 255, 255],
       [366, 366, 366, ..., 385, 385, 230],
       ...,
       [194, 194, 194, ..., 366, 366, 366],
       [367, 477, 477, ..., 380, 229, 229],
       [395, 403, 403, ..., 143, 143, 345]])>, <tf.Tensor: shape=(20, 500), dtype=int64, numpy=
array([[380, 229, 229, ...,  23,  22, 171],
       [230, 230, 230, ..., 255, 255, 394],
       [366, 366, 366, ..., 385, 230, 230],
       ...,
       [194, 194, 194, ..., 366, 366, 366],
       [477, 477, 475, ..., 229, 229, 381],
       [403, 403, 483, ..., 143, 345, 350]])>)


In [7]:
#  The vocabulary length in characterrs
vocabulary_length = len(vocabulary)

# The embedding dimension
embedding_dimension = 512

# Number of RNN units
recurrent_nn_units = 1024

In [8]:
if tf.test.is_gpu_available():
    recurrent_nn = tf.compat.v1.keras.layers.CuDNNGRU
    print("GPU in use")
else:
    import functools
    recurrent_nn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')
    print("CPU in use")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU in use


In [9]:
def build_model(vocabulary_size, embedding_dimension, recurrent_nn_units, batch_size):
    model = tf.keras.Sequential(
        [tf.keras.layers.Embedding(vocabulary_size, embedding_dimension, batch_input_shape=[batch_size, None]),
    recurrent_nn(recurrent_nn_units, return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True),
    tf.keras.layers.Dense(vocabulary_length)
  ])
    return model

In [59]:
model = build_model(
  vocabulary_size = len(vocabulary),
  embedding_dimension=embedding_dimension,
  recurrent_nn_units=recurrent_nn_units,
  batch_size=batch)

In [60]:
print(dataset.take(10))
for input_example, target_example in dataset.take(1):
    print ('Input data: ', input_example)
    print ('Target data:', target_example)

<TakeDataset shapes: ((20, 500), (20, 500)), types: (tf.int64, tf.int64)>
Input data:  tf.Tensor(
[[198 204  52 ... 185 185  35]
 [ 32  24  39 ... 366 366 363]
 [381 381 381 ... 206 362 204]
 ...
 [366 366 366 ... 385 385 230]
 [210 207 204 ... 224 227 224]
 [480 380 229 ... 172  23  22]], shape=(20, 500), dtype=int64)
Target data: tf.Tensor(
[[204  52 215 ... 185  35  35]
 [ 24  39  32 ... 366 363 366]
 [381 381 381 ... 362 204 200]
 ...
 [366 366 366 ... 385 230 230]
 [207 204 204 ... 227 224 227]
 [380 229 229 ...  23  22 171]], shape=(20, 500), dtype=int64)


In [61]:
print(dataset.take(1))
for batch_input_example, batch_target_example in dataset.take(1):
    print(batch_input_example)
    batch_predictions_example = model(batch_input_example)
    print(batch_predictions_example.shape, "# (batch, sequence_length, vocabulary_length)")

<TakeDataset shapes: ((20, 500), (20, 500)), types: (tf.int64, tf.int64)>
tf.Tensor(
[[237  82 237 ... 201 201 201]
 [230  77 230 ... 230 230 230]
 [230 237 230 ... 381 381 381]
 ...
 [381 381 381 ... 206 362 204]
 [385 385 381 ... 233 380 478]
 [227 224  74 ... 230  77 228]], shape=(20, 500), dtype=int64)
(20, 500, 512) # (batch, sequence_length, vocabulary_length)


In [62]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (20, None, 512)           262144    
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (20, None, 1024)          4724736   
_________________________________________________________________
dense_4 (Dense)              (20, None, 512)           524800    
Total params: 5,511,680
Trainable params: 5,511,680
Non-trainable params: 0
_________________________________________________________________


In [63]:
sampled_indices = tf.random.categorical(logits=batch_predictions_example[0], num_samples=1)

sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [64]:
sampled_indices

array([289, 309,  45, 358, 218, 180, 168, 350,  18, 411, 180, 427, 312,
       404,  30, 223, 343, 130, 478, 371,  52, 317, 418, 453, 109, 510,
       185, 343, 330, 114, 136,  69, 181, 488, 440, 331, 142, 498,  87,
       336, 274, 429, 160, 353, 216, 177,  10, 415, 181, 264, 398,  65,
       328, 145, 409,  20,  17, 161, 498,  35, 417,  38,  12,  53,  33,
        68,  30, 340, 189,  62, 275, 477, 111,  99, 350, 455, 412,  44,
       158, 308, 370, 444,  72, 187, 184, 334, 231, 506, 300,  49, 465,
       317,   5, 409,  39, 451, 509, 510, 425, 278, 246,  43, 291, 263,
       108, 335, 160, 510, 162, 181, 124, 410, 429, 463, 191, 344, 383,
       390, 312, 316, 336, 252,  65,  12,  66, 360, 319, 484, 404, 497,
        87, 444, 214, 438, 186, 261, 189, 285,  62, 479, 160, 248, 301,
       265, 340, 198, 248, 217, 225, 450, 144, 447, 164, 494, 309, 366,
        42, 388, 198, 289,  67,  83, 446, 230, 278,  55, 434, 168, 111,
       101, 395,  73, 470, 184, 444,  39, 438,  77,  24, 310, 33

In [65]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [66]:
batch_loss_example  = tf.compat.v1.losses.sparse_softmax_cross_entropy(batch_target_example, batch_predictions_example)
print("Prediction shape: ", batch_predictions_example.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", batch_loss_example.numpy())

Prediction shape:  (20, 500, 512)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       6.2385006


In [67]:
#next produced by upgrade script.... 
#model.compile(optimizer = tf.compat.v1.train.AdamOptimizer(), loss = loss) 
#.... but following optimizer is available.
model.compile(optimizer = tf.optimizers.Adam(), loss = loss)

In [68]:
# Directory where the checkpoints will be saved
directory = './checkpoints-copy'
# Name of the checkpoint files
file_prefix = os.path.join(directory, "ckpt_{epoch}")

callback=[tf.keras.callbacks.ModelCheckpoint(filepath=file_prefix, save_weights_only=True)]


In [69]:
epochs=500

In [70]:
history = model.fit(dataset, epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=callback)

Epoch 1/500
1/1 [==============================] - 0s 451ms/step - loss: 6.2385
Epoch 2/500
1/1 [==============================] - 0s 74ms/step - loss: 6.1467
Epoch 3/500
1/1 [==============================] - 0s 74ms/step - loss: 6.0348
Epoch 4/500
1/1 [==============================] - 0s 76ms/step - loss: 5.6879
Epoch 5/500
1/1 [==============================] - 0s 73ms/step - loss: 14.8114
Epoch 6/500
1/1 [==============================] - 0s 72ms/step - loss: 5.3257
Epoch 7/500
1/1 [==============================] - 0s 69ms/step - loss: 5.4326
Epoch 8/500
1/1 [==============================] - 0s 72ms/step - loss: 5.4660
Epoch 9/500
1/1 [==============================] - 0s 69ms/step - loss: 5.4715
Epoch 10/500
1/1 [==============================] - 0s 72ms/step - loss: 5.3514
Epoch 11/500
1/1 [==============================] - 0s 69ms/step - loss: 5.2778
Epoch 12/500
1/1 [==============================] - 0s 72ms/step - loss: 5.0558
Epoch 13/500
1/1 [=============================

1/1 [==============================] - 0s 71ms/step - loss: 0.1550
Epoch 205/500
1/1 [==============================] - 0s 74ms/step - loss: 0.1627
Epoch 206/500
1/1 [==============================] - 0s 73ms/step - loss: 0.1407
Epoch 207/500
1/1 [==============================] - 0s 73ms/step - loss: 0.1468
Epoch 208/500
1/1 [==============================] - 0s 71ms/step - loss: 0.1498
Epoch 209/500
1/1 [==============================] - 0s 80ms/step - loss: 0.1132
Epoch 210/500
1/1 [==============================] - 0s 76ms/step - loss: 0.1397
Epoch 211/500
1/1 [==============================] - 0s 77ms/step - loss: 0.1454
Epoch 212/500
1/1 [==============================] - 0s 73ms/step - loss: 0.1359
Epoch 213/500
1/1 [==============================] - 0s 75ms/step - loss: 0.1372
Epoch 214/500
1/1 [==============================] - 0s 74ms/step - loss: 0.1311
Epoch 215/500
1/1 [==============================] - 0s 73ms/step - loss: 0.1288
Epoch 216/500
1/1 [=======================

1/1 [==============================] - 0s 73ms/step - loss: 0.0207
Epoch 407/500
1/1 [==============================] - 0s 71ms/step - loss: 0.0189
Epoch 408/500
1/1 [==============================] - 0s 71ms/step - loss: 0.0203
Epoch 409/500
1/1 [==============================] - 0s 73ms/step - loss: 0.0207
Epoch 410/500
1/1 [==============================] - 0s 71ms/step - loss: 0.0224
Epoch 411/500
1/1 [==============================] - 0s 74ms/step - loss: 0.0197
Epoch 412/500
1/1 [==============================] - 0s 73ms/step - loss: 0.0198
Epoch 413/500
1/1 [==============================] - 0s 73ms/step - loss: 0.0201
Epoch 414/500
1/1 [==============================] - 0s 73ms/step - loss: 0.0204
Epoch 415/500
1/1 [==============================] - 0s 71ms/step - loss: 0.0181
Epoch 416/500
1/1 [==============================] - 0s 73ms/step - loss: 0.0207
Epoch 417/500
1/1 [==============================] - 0s 71ms/step - loss: 0.0194
Epoch 418/500
1/1 [=======================

In [85]:
for layer in model.layers[1:2]:
    weights = layer.get_weights() 
    for s in weights:
        print(s.shape)
model.summary()

(512, 3072)
(1024, 3072)
(6144,)
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (20, None, 512)           262144    
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (20, None, 1024)          4724736   
_________________________________________________________________
dense_4 (Dense)              (20, None, 512)           524800    
Total params: 5,511,680
Trainable params: 5,511,680
Non-trainable params: 0
_________________________________________________________________


In [40]:
tf.train.latest_checkpoint(directory)

'./checkpoints-copy/ckpt_500'

In [89]:
model = build_model(vocabulary_size, embedding_dimension, recurrent_nn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(directory))

model.build(tf.TensorShape([1, None]))

In [90]:
for layer in model.layers[1:2]:
    weights = layer.get_weights() 
    for s in weights:
        print(s.shape)
model.summary()

(512, 3072)
(1024, 3072)
(6144,)
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 512)            262144    
_________________________________________________________________
cu_dnngru_6 (CuDNNGRU)       (1, None, 1024)           4724736   
_________________________________________________________________
dense_6 (Dense)              (1, None, 512)            524800    
Total params: 5,511,680
Trainable params: 5,511,680
Non-trainable params: 0
_________________________________________________________________


In [47]:
def generate_text(model, start_string, temperature, characters_to_generate):

  # Vectorise  start string into numbers
    input_string = [c2emb[code] for code in start_string]
    input_string = tf.expand_dims(input_string, 0)

  # Empty string to store  generated text
    generated = start_string
    
  # (Batch size is 1)
    model.reset_states()
    for i in range(characters_to_generate):
        predictions = model(input_string)
        print(input_string.shape)
      # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(logits=predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted word as the next input to the model
      # along with the previous hidden state
        input_string = tf.expand_dims([predicted_id], 0)

        generated.append(emb2c[predicted_id])

    return generated # generated is a list

In [48]:
generated_text = generate_text(model=model, start_string=[text[np.random.randint(len(text))]], temperature=0.1, characters_to_generate = 12000)
print(generated_text)

(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)

(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)

(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)


KeyboardInterrupt: 

In [ ]:
np.save("fish_murme2.npy", generated_text)

In [ ]:
np.random.randint(len(text))